# Claims Matching Rules
1. Carrier excel file(s) must be in .xlsx format (if .xls, you must convert to .xlsx)
2. This Python file must be in the same folder as your data
3. You can create a copy of this file and paste into your client folder where carrier data is located
4. If this is your first time using this code, remove # from the below installer, run code
5. If you have multiple sheets in 1 workbook, enter them in as if they were seperate requests



# First Time User - Install 

In [54]:
#To install the below, remove the hashtag from all lines of code and hit run
# If you are NOT a first time user, skip this code and proceed to Step 1

# !pip install xlsxwriter
# !pip install xlutils
# !pip install xlrd
# !pip install xlwt
# !pip install openpyxl
# !pip install easygui
# !pip install pillow
# !pip install formlayout
# !pip install PySide2
# !pip install wxpython
# !pip install dash
# !pip install pysimplegui
# !pip install pyxlsb
# !pip3 install pyxlsb
#!pip install --upgrade pip
#!pip install joblib
#!pip install sklearn
# !pip install PyInstaller


# Download libraries

In [84]:
print(' ')
print('***************************************')
print("               WELCOME                  ")
print('***************************************')
print(' ')
print("---------------------------------------")
print("Medical/Rx Cliams Matching Tool (CMT)")
print("---------------------------------------")
print(" ")
print(" ")
print("             Instructions            ")
print('=======================================')
print("     1. Review ALL files & identify which column(s) you will be matching on")
print("     2. CLOSE ALL files prior to start")
print("     3. DO NOT use original carrier files with code, make a copy")
print("     4. Create a working folder which includes your working files ")
print("     5. Working folder should contain ONLY files you need matched")
print('')
print("             Matching Options        ")
print('=======================================')
print("     1. Single Match")
print("        - You have 1 identifier i.e. SSN, Member ID, Claimant name in same format")
print("     2. Name Match & Name Match w/ DOB")
print("        - You have varying name formats")
print("        - You want to match on name + DOB")
print("")

 
***************************************
               WELCOME                  
***************************************
 
---------------------------------------
Medical/Rx Cliams Matching Tool (CMT)
---------------------------------------
 
 
             Instructions            
     1. Review ALL files & identify which column(s) you will be matching on
     2. CLOSE ALL files prior to start
     3. DO NOT use original carrier files with code, make a copy
     4. Create a working folder which includes your working files 
     5. Working folder should contain ONLY files you need matched

             Matching Options        
     1. Single Match
        - You have 1 identifier i.e. SSN, Member ID, Claimant name in same format
     2. Name Match & Name Match w/ DOB
        - You have varying name formats
        - You want to match on name + DOB



In [98]:
## from os.path import expanduser as ospath
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from datetime import date
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
import joblib
from xlrd import open_workbook
import re
import xlrd
from sklearn import model_selection, preprocessing
from pandas.api.types import is_numeric_dtype
import os
import glob
import re
import time
from sklearn import model_selection, preprocessing, feature_extraction
import sklearn.utils._cython_blas


# Defined Function

In [99]:
def colnum_string(n): 
    string=''
    while n>0: 
        n, remainder=divmod(n-1, 26)
        string=chr(65+remainder)+string
    return string

# GLOB - pull in all file in folder


In [100]:

start = time.time()

file_path=input('Please copy/paste the directory path: ')
os.chdir = file_path

files_closed = 2
while not files_closed == 1:
    files_closed = int(input("Please close all files in folder. Ready to move on? Yes=1, No=2 "))

#-----------------------GLOB XLS--------------------------
# ##check all files with .xls in folder and return file name
print('Folder analysis start...')
print ("")
print('Running analysis for .xls files in folder .xls files must be converted to .xlsx ')
print ("---")



files_xls = glob.glob(f'{file_path}\*.xls')
for folderfiles in files_xls:
    print(f'Filename: {folderfiles}') 
    print ("")
    print ("---")

#-----------------------GLOB XLSB--------------------------

#check all files with .xlsb in folder and return file name
print('Running analysis for .xlsb files in folder .xlsb files must be converted to .xlsb')
print ("---")
files_xlsb = glob.glob(f'{file_path}\*.xlsb')
for folderfiles in files_xlsb:
    print(f'Filename: {folderfiles}') 
    print ("")
    print ("---")



print('Running analysis for .xlsx files folder this could take a few mintues, please wait')
print ("---")

#-----------------------GLOB XLSX--------------------------
pd.set_option('display.float_format', lambda x: '%.5f' % x)

count=0
file_selection={}
files_xlsx = glob.glob(f'{file_path}\*.xlsx')
files_xlsx=[each_file for each_file in files_xlsx if not each_file[0:2] == '~$']
for each_file in files_xlsx:
    with open_workbook(each_file, on_demand=True) as book:
        sheets=book.sheet_names()
        #GO INTO EACH SHEET
        for each_sheet in sheets: 
            new_file={}
            count+=1
            print("+----------------------------------------------------+")
            print(f'{count}: File: {each_file} :: Sheet: {each_sheet}')
            new_file['file']=each_file
            new_file['sheet']=each_sheet
            file_selection[count]=new_file


print ("")
print ("---")
print('Folder analysis complete...')


Please copy/paste the directory path: C:\Users\a0766635\Desktop\Python\Cherokee\data
Please close all files in folder. Ready to move on? Yes=1, No=2 1
Folder analysis start...

Running analysis for .xls files in folder .xls files must be converted to .xlsx 
---
Running analysis for .xlsb files in folder .xlsb files must be converted to .xlsb
---
Running analysis for .xlsx files folder this could take a few mintues, please wait
---
+----------------------------------------------------+
1: File: C:\Users\a0766635\Desktop\Python\Cherokee\data\Cherokee Nation Businesses - RXT1210DM - Claim Detail Report - YTD 08-2019.xlsx :: Sheet: Large Claims
+----------------------------------------------------+
2: File: C:\Users\a0766635\Desktop\Python\Cherokee\data\Cherokee Nation Businesses - RXT1210DM - Claim Detail Report - YTD 08-2019.xlsx :: Sheet: LC over 25k
+----------------------------------------------------+
3: File: C:\Users\a0766635\Desktop\Python\Cherokee\data\Cherokee Nation Businesses 

['C:\\Users\\a0766635\\Desktop\\Python\\Cherokee\\Cherokee Nation Businesses - RXT1210DM - Claim Detail Report - YTD 08-2019.xlsx',
 'C:\\Users\\a0766635\\Desktop\\Python\\Cherokee\\~$NonFuzzy.xlsx']

# How many sheets do we match on, and which sheets?

In [101]:
## creating an empty list for inputs needed to identify key elements for the file
confirm = 2
while not confirm ==1:
    
    filename = []
    sheetname = []
    dfname = []
    ## Clear any exisiting list items for fresh start
    filename.clear()
    sheetname.clear()
    dfname.clear()


    n = int(input('Input # of sheets to merge: i.e. 3 ' ))


    ##iterating
    for i in range(0, n): 
        selection=int(input('Input # file/sheet index: '))
        ele1 = file_selection[selection]['file']
        ele2 = file_selection[selection]['sheet']
        ele3 = f'df{i}'
        filename.append(ele1) ## adding the element 
        sheetname.append(ele2) ## adding the element
        dfname.append(ele3)
    print("")
    
    
#     print(f'You have selected the following files {filename}')
#     print(f'You have selected the following sheet {sheetname}')
    confirm = int(input("Ready to move on? Yes=1, No=2: "))
    print("")


Input # of sheets to merge: i.e. 3 3
Input # file/sheet index: 4
Input # file/sheet index: 6
Input # file/sheet index: 9

Ready to move on? Yes=1, No=2: 1



# Ask for row # where header should start (add machine learning here)

In [ ]:
#pd.set_option('display.float_format', lambda x: '%.5f' % x)
##Machine learning
test_texts=[]
row_nums=[]
row_nums.clear()
headerrow = []
headerrow.clear
exceldict = {}
header_row_num_ML=[]
exceldict.clear()
df={}

Correct_header=2
confirm = 2
while not confirm ==1:
    for i in range(len(filename)):
        with open_workbook(filename[i], on_demand=True) as book:
            sheet=book.sheet_by_name(sheetname[i])
            try:
                for each_row in range(min(sheet.nrows, 20)):
                    text=sheet.row_values(each_row)[0:]
                    text_stitch=' '.join(str(cell) for cell in text if re.search('[a-zA-Z]', str(cell)))
                    if not text_stitch=='': 
                        row_nums.append(each_row)
                        test_texts.append(text_stitch)
                #print(test_texts)
                df[dfname[i]]=pd.DataFrame()
                df[dfname[i]]['text']=test_texts
                df[dfname[i]]['row_num']=row_nums  
                count_vectorizer=joblib.load('Count_vectorizer.sav')
                model=joblib.load('LR_model.sav')
                test_vector=count_vectorizer.transform(df[dfname[i]]['text'])
                df[dfname[i]]['label']=model.predict(test_vector)
                header_row_num=df[dfname[i]][df[dfname[i]]['label']==1]['row_num'].values[0]
                print(" ")
                print("----IDENTIFYING HEADER(auto)----")
                print(f"For {filename[i]} :: {sheetname[i]}:")
                print(f'  System identified header on row {header_row_num+1}')
                print(" ")
                df1 = pd.read_excel(f'{filename[i]}', (sheetname[i]), header=header_row_num, dtype={'column': object})
                #print(f'For {filename[i]}, {sheetname[i]} system identified header at excel row {header_row_num_ML+1}')
                print (list(df1.head(0)))
                Correct_header=int(input("Correct header row? 1=Yes, 2=No "))
                print("Processing...")

                if Correct_header == 1:
                    headerrow.append(header_row_num)
                    exceldict[dfname[i]] = pd.read_excel(f'{filename[i]}', (sheetname[i]), header=headerrow[i], dtype={'column': object})
                    test_texts.clear()
                    row_nums.clear()
                else:
                    print("----IDENTIFYING HEADER(else)----")
                    print(f'For {filename[i]} :: {sheetname[i]}:' )
                    print(" ")
                    print(pd.read_excel(f'{filename[i]}', (sheetname[i]), header=None).iloc[:15, :2])
                    #exceldict[dfname[i]] = pd.read_excel(f'{filename[i]}', (sheetname[i]), header=None, dtype={'column': object})
                    #print(exceldict[dfname[i]].iloc[:18, :2])
                    header_num = int(input("Review in left column, what row # does your header start in? "))
                    print("Processing...")
                    headerrow.append(header_num)
                    exceldict[dfname[i]] = pd.read_excel(f'{filename[i]}', (sheetname[i]), header=headerrow[i], dtype={'column': object}, inplace=True)
                    test_texts.clear()
                    row_nums.clear()
            except:
                print(" ")
                print("----IMPORTANT: PLEASE SEND COPY OF THIS FILE TO SANDY.LUCEV@AON.COM----")
                print("----IDENTIFYING HEADER(except)----")
                print(f'For {filename[i]} :: {sheetname[i]}:' )
                print(" ")
                print(pd.read_excel(f'{filename[i]}', (sheetname[i]), header=None).iloc[:15, :2])
                #exceldict[dfname[i]] = pd.read_excel(f'{filename[i]}', (sheetname[i]), header=None, dtype={'column': object})
                #print(exceldict[dfname[i]].iloc[:18, :2])
                header_num = int(input("Review in left column, what row # does your header start in? "))
                headerrow.append(header_num)
                exceldict[dfname[i]] = pd.read_excel(f'{filename[i]}', (sheetname[i]), header=headerrow[i], dtype={'column': object}, inplace=True)
                test_texts.clear()
                row_nums.clear()
    confirm = int(input("Ready to move on? Yes=1, No=2: "))

 
----IDENTIFYING HEADER(auto)----
For C:\Users\a0766635\Desktop\Python\Cherokee\data\Cherokee Nation Businesses - RXT1210DM - Claim Detail Report - YTD 08-2019.xlsx :: Claim Detail Report Page_1:
  System identified header on row 15
 
['Carrier ID', 'Carrier Name', 'Account ID', 'Account Name', 'Group ID', 'Group Name', 'Plan Code', 'Member ID', 'Member Last Name', 'Member First Name', 'Last, First', 'Member Middle Initial', 'Member Gender', 'Member Date of Birth', 'Member Address 1', 'Member Address 2', 'Member City', 'Member State', 'Member Zip', 'Member Phone', 'Date Filled', 'Incurred Month', 'Date Submitted', 'Paid Month', 'RxCLAIM Number', 'RxCLAIM Sequence Number', 'RxCLAIM Status', 'Rx Number', 'Refill Number', 'Claim Origin Code', 'Prescriber Submitted ID', 'Prescriber DEA ID', 'Prescriber NPI', 'Prescriber Last Name', 'Prescriber First Name', 'Prescriber Middle Initial', 'Prescriber Degree', 'Prescriber Address 1', 'Prescriber Address 2', 'Prescriber City', 'Prescriber State

# Pick matching id from the selected headers.
##what if the header i want to match on has a different name i.e. Member ID / Mem ID? 

In [91]:
# ##here you identify which column header you will be matching all the files on
print("Processing: identifying match criteria")
print ("Processing: This may take a few minutes... ")
Combined_df_list = []
Match_Type = 'str' 
Match_ID_ColNum =[]
Match_ID = []
confirm =2
while not confirm ==1:
    print(" ")
    for i in range(0, n): 
        try: 
            print(" ")
            Matching_Options=list(exceldict[dfname[i]].columns)
            print("----IDENTIFYING MATCH COLUMN----")
            print(f"  Review header columns for: {filename[i]}, {sheetname[i]}")
            print(" ")
            for index, cols in enumerate(Matching_Options):
                print (f'  {index}: {cols}')
            print(" ")
            print("")
            match = int(input("Select Index # to match: ")) 
            #dfname[i]["_Selected Match"]=dfname[i][[Matching_Options[match].str]]
            Match_ID.append(Matching_Options[match]) ## adding the element 
            Match_ID_ColNum.append(match)
            print("--------------------------")
    #         print(f'Confirm no quoations: you selected: {Match_ID[i]}')
            exceldict[dfname[i]]["_Match"]=exceldict[dfname[i]][Matching_Options[match]].astype(dtype= "str").replace('\.0', '', regex=True)
            get_unique=exceldict[dfname[i]]["_Match"]
            for i in get_unique: 
                if i not in Combined_df_list: 
                    Combined_df_list.append(i)
        except:
            print("Error - please restart your file")
    confirm = int(input("Ready to move on? Yes=1, No=2: "))
    


Processing: identifying match criteria
Processing: This may take a few minutes... 
 
 
----IDENTIFYING MATCH COLUMN----
  Review header columns for: C:\Users\a0766635\Desktop\Python\Cigna Med Rx 1\data\CIGNA - Med & Rx2017onDesk.xlsx, 17-18-1
 
  0: BRANCH
  1: FAT
  2: RAT
  3: MEMBER ID
  4: EMPLOYEE ID
  5: SUBSCRIBER NAME
  6: REL
  7: GENDER
  8: AGE BAND
  9: ICD CODE
  10: ICD DESCRIPTION
  11: ICD VERSION
  12: DRUG CLAIMS
  13: PAID CLAIMS
  14: CLAIMANT TOTAL
  15: Unnamed: 15
  16: Unnamed: 16
  17: Unnamed: 17
  18: Unnamed: 18
  19: Unnamed: 19
  20: Unnamed: 20
  21: Unnamed: 21
  22: Unnamed: 22
  23: Unnamed: 23
  24: Claimant ID
  25: Unique Claimant ID
  26: Unique Claimant ID.1
  27: Unique Claimant ID.2
  28: Unique Claimant ID.3
  29: Unique Claimant ID.4
  30: Unique Claimant ID.5
 

Select Index # to match: 4
--------------------------
 
----IDENTIFYING MATCH COLUMN----
  Review header columns for: C:\Users\a0766635\Desktop\Python\Cigna Med Rx 1\data\CIGNA - Med 

In [92]:
#Combined_vlookup_table['_Match'].apply(lambda x: '%.0f' % x).values.tolist()
#pd.options.display.float_format = '{:.2f}'.format
Combined_vlookup_table = pd.DataFrame(Combined_df_list)
Combined_vlookup_table.rename(columns={0:'_Match'}, inplace=True )
Combined_vlookup_table.dropna(how='any', inplace=True)
Combined_vlookup_table['_Match'].replace(" ", np.nan, inplace=True)
Combined_vlookup_table['_Match'].replace("nan", np.nan, inplace=True)
Combined_vlookup_table.dropna(inplace=True)
Combined_vlookup_table.sort_values("_Match", inplace = True)
#pd.set_option('display.max_rows', Combined_vlookup_table.shape[0]+1)
#print(Combined_vlookup_table)
#Combined_vlookup_table.dtypes
#print("Processing: successful match")

                  _Match
74           00221011701
83           00804723201
84           01008801901
95           01076862001
47       044484444723201
86           04886663301
14         0585433323201
4            06166689901
103          06999903801
75           08990809701
104          09070443101
87           09097987001
101          09310214801
82           09322633801
81           09414620301
96           10800143401
89           11072737701
27       113335003331201
54         1144472737701
21   1153332333333333301
92           11700071201
60         1174440071201
93           12404106901
61         1244444106901
28           12454106901
79           12660310101
43         1266031444101
9            12660315101
64       144484440143401
31           15850143401
29         2169423338501
94           21694278001
62         2169427844401
37         2309393334101
69           23093974101
20     23163336533333301
88           23167607701
53         2316764447701
7            23639680401


# Show the combined df's and ask if any lines need to be deleted

In [93]:
print("Processing: creating unique claimant #s ")
# ##Creates new column "UniqueClaimant" 
ClaimantID = [f'Claimant {i}' for i in range(1,1+len(Combined_vlookup_table))]

# ##this adds ClaimantID to the combined dataframe and names if Unique Claimant
Combined_vlookup_table["UniqueClaimant"] = ClaimantID
Combined_vlookup_table.columns.name=None
#Combined_vlookup_table
#print("Processing: unique claimant #s created")

# Combined_vlookup_table.dtypes

Processing: creating unique claimant #s 


# Create vlookup tables in each file and outside file

In [96]:
os.chdir = file_path
from openpyxl import Workbook
from openpyxl import load_workbook

# ##creates date to add to end of file
# today = date.today()

# ##exports vlookup table into same folder as the file and creates two tabs, 1 for just unique claimant and other with all columns
# Combined_vlookup_table.to_excel (f'_Automated_Lookup{today}.xlsx', sheet_name='Automated_Lookup', index = False, header=True,)
# #Combined_df.to_excel (f'_Automated_Full Report{today}.xlsx', sheet_name='Automated_all_col', index = False, header=True,)

# print('Processing: vlookup sheet in folder created')
# close_file = 2
# while not close_file == 1:
#     close_file = int(input("Are your xlsx files closed? 1=Yes, 2=No: "))
#     if close_file == 1:
#         print('Processing: creating vlookup sheet in exisiting excel')
#         ##exports vlookup table into original file
#         for i in range(len(exceldict)):
#             path = filename[i]
#             with pd.ExcelWriter(path, engine='openpyxl') as writer:
#                 wb = openpyxl.load_workbook(filename = path)
#                     wb = openpyxl.load_workbook(filename = filename[i])
#     sheet = wb[sheetname[i]]
# #                 writer.wb = book
# #                 writer.sheets = dict((ws.title,ws) for ws in book.worksheets)
#                 sheet = wb[sheetname[i]]
#                 #sheet.cell(row= headerrow[i]+1, column=len(exceldict[key].columns)+1).fill = PatternFill(start_color="98ea64", end_color="98ea64", fill_type = "solid")
#                 Combined_vlookup_table.to_excel(path, sheet_name= f'Automated_Lookup', header=True, index=False)
#                 sheet=book["Automated_Lookup"]
#                 sheet.sheet_properties.tabColor = 'FF0000'
#                 book.save(filename[i])
#                 writer.save()
#     else:
#         pass
# print("")
# print('Processing: vlookup sheet in exisiting excel created')
print ("Processing: creating vlookup sheets in folder")

##creates date to add to end of file
today = date.today()

##exports vlookup table into same folder as the file and creates two tabs, 1 for just unique claimant and other with all columns
lookup = '/Automated_Lookup{0}.xlsx'.format(today)
Combined_vlookup_table.to_excel (file_path + lookup, sheet_name='Automated_Lookup', index = False, header=True,)
#Combined_df.to_excel (f'_Automated_Full Report{today}.xlsx', sheet_name='Automated_all_col', index = False, header=True,)

print('Processing: vlookup sheet in folder created')
close_file = 2
while not close_file == 1:
    close_file = int(input("Are your xlsx files closed? 1=Yes, 2=No: "))
    if close_file == 1:
        print('Processing: creating vlookup sheet in exisiting excel')
        ##exports vlookup table into original file
        for i in range(len(exceldict)):
            print(f"Processing: {sheetname[i]}")
            book = load_workbook(filename[i])
            writer = pd.ExcelWriter(filename[i], engine='openpyxl')
            writer.book = book
            writer.sheets = dict((ws.title,ws) for ws in book.worksheets)
#                 sheet = writer.book[sheetname[i]]
                #sheet.cell(row= headerrow[i]+1, column=len(exceldict[key].columns)+1).fill = PatternFill(start_color="98ea64", end_color="98ea64", fill_type = "solid")
            Combined_vlookup_table.to_excel(writer, sheet_name= f'Automated_Lookup', header=True, index=False)
            sheet=book["Automated_Lookup"]
            sheet.sheet_properties.tabColor = 'FF0000'
#                 book.save(filename[i])
            writer.save()
    else:
        pass
print("")
print('Processing: vlookup sheet in exisiting excel created')

Processing: creating vlookup sheets in folder
Processing: vlookup sheet in folder created
Are your xlsx files closed? 1=Yes, 2=No: 1
Processing: creating vlookup sheet in exisiting excel
Processing: 17-18-1
Processing: 18-19-1
Processing: 18-19-1

Processing: vlookup sheet in exisiting excel created


In [83]:
writer

'C:\\Users\\a0766635\\Desktop\\Python\\TESTING ROOM\\data\\Test - Med & Rx2017.xlsx'

#  Add Vlookup formula to the excel
- Need to make the column that you are referencing auto turn to "text" and not "general"
- need to figure out how to get coordinates to the columns and row for referencing and writing vlookup

In [97]:
for i in range(len(exceldict)):
    wb = openpyxl.load_workbook(filename = filename[i])
    sheet = wb[sheetname[i]]

    print("----MATCHING COLUMNS IN EXCEL----")
    print (f'Analyzing file: {filename[i]} :: sheet {sheetname[i]} dimensions {sheet.dimensions}')
    row_max_count = sheet.max_row
    row_min_count = sheet.min_row
    column_max_count = sheet.max_column
    column_min_count = sheet.min_column
    col_min = colnum_string(column_min_count)
    col_max = colnum_string(column_max_count+1)
    col_match_letter = colnum_string(Match_ID_ColNum[i]+1)
    startrow = headerrow[i]+1


    print(f' Processing: begin row {col_match_letter}{startrow}, end row {col_match_letter}{row_max_count}')
    print(" ")
    for row in sheet[f'{col_max}{startrow+1}:{col_max}{row_max_count}']:
        for cell in row:
            lookup_value=sheet[f'{col_match_letter}{cell.row}'].value
            # check if lookup_value is blank
            if lookup_value:
                if is_numeric_dtype(exceldict[dfname[i]][Match_ID[i]])== True:
                    cell.value = "=VLOOKUP({0}, 'Automated_Lookup'!A:B, 2, FALSE)".format('"'+str(lookup_value).lstrip('0')+'"')
                else:
                    cell.value = "=VLOOKUP({0}, 'Automated_Lookup'!A:B, 2, FALSE)".format('"'+str(lookup_value)+'"')
                    
    for row in sheet[f'{col_max}{startrow}:{col_max}{startrow}']:
        for cell in row:
            lookup_value=sheet[f'{col_match_letter}{cell.row}'].value
            # check if lookup_value is blank
            if lookup_value:
                cell.value = "Unique Claimant ID"
                sheet.cell(row= startrow, column=column_max_count+1).fill = PatternFill(start_color="98ea64", end_color="98ea64", fill_type = "solid")

                
            
#             cell.value = "=VLOOKUP(TEXT(VALUE({0}{1}),0), 'Automated_Lookup'!A:B, 2, FALSE)".format(col_match_letter, cell.row)
            #cell.value = "=IF(VLOOKUP({0}{1}, 'Automated_Lookup'!A:B, 2)="", "",VLOOKUP({0}{1}, 'Automated_Lookup'!A:B, 2))".format(col_match_letter, cell.row)
    
        #sheet.cell(row= row_min_count, column= column_max_count.fill = PatternFill(start_color="98ea64", end_color="98ea64", fill_type = "solid")

               
#     print(f' Processing: begin row {col_match_letter}{startrow}, end row {col_match_letter}{row_max_count}')
#     print(" ")
#     for row in sheet[f'{col_max}{startrow+1}:{col_max}{row_max_count}']:
#         for cell in row:
#             lookup_value=sheet[f'{col_match_letter}{cell.row}'].value
#             # check if lookup_value is blank
#             if lookup_value:
#                 #cell.value = "=VLOOKUP({0}, 'Automated_Lookup'!A:B, 2, FALSE)".format('"'+str(lookup_value).lstrip('0')+'"')
#                 cell.value = "=VLOOKUP({0}, 'Automated_Lookup'!A:B, 2, FALSE)".format('"'+str(lookup_value)+'"')
# #             cell.value = "=VLOOKUP(TEXT(VALUE({0}{1}),0), 'Automated_Lookup'!A:B, 2, FALSE)".format(col_match_letter, cell.row)
#             #cell.value = "=IF(VLOOKUP({0}{1}, 'Automated_Lookup'!A:B, 2)="", "",VLOOKUP({0}{1}, 'Automated_Lookup'!A:B, 2))".format(col_match_letter, cell.row)
#             #         sheet.cell(row= headerrow[i]+1, column=1+len(exceldict[key].columns)).fill = PatternFill(start_color="98ea64", end_color="98ea64", fill_type = "solid")

    
    wb.save(filename[i])

    print("-------")
print("")
print("")
print ("Finished. Please check your file for vlookup tab and formulas in main sheets")
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Code run time: {:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
print("")
print("")
os.system("done")

----MATCHING COLUMNS IN EXCEL----
Analyzing file: C:\Users\a0766635\Desktop\Python\Cigna Med Rx 1\data\CIGNA - Med & Rx2017onDesk.xlsx :: sheet 17-18-1 dimensions A5:AF101
 Processing: begin row E12, end row E101
 
-------
----MATCHING COLUMNS IN EXCEL----
Analyzing file: C:\Users\a0766635\Desktop\Python\Cigna Med Rx 1\data\CIGNA - Med & Rx2018.xlsx :: sheet 18-19-1 dimensions A5:R94
 Processing: begin row E12, end row E94
 
-------
----MATCHING COLUMNS IN EXCEL----
Analyzing file: C:\Users\a0766635\Desktop\Python\Cigna Med Rx 1\data\CIGNA - Med & Rx2019.xlsx :: sheet 18-19-1 dimensions A5:Q109
 Processing: begin row E12, end row E109
 
-------


Finished. Please check your file for vlookup tab and formulas in main sheets
Code run time: 00:02:10.90




1

# END CODE